In [1]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from utils import flat, trace
from tqdm import tqdm
import psutil
import torch
import gc
from itertools import product
from multiprocessing import Pool
from tqdm import tqdm
from data import RFCXDataset
from model import kaggle_metric
from utils import sigmoid

In [2]:
data_dir = '../../input/kaggle/train'
fnames = os.listdir(data_dir)
recording_ids = [x[:-5] for x in fnames]
print(len(fnames))
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
sample_submission = pd.read_csv('../../input/kaggle/sample_submission.csv')
folds = joblib.load('folds.jl')
print(train_tp.shape)
display(train_tp.head())
print(train_fp.shape)
display(train_fp.head())

4729
(1216, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70


(7781, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [7]:
sub = pd.read_csv('submission.csv', index_col='recording_id')
display(sub)
display(sub.describe())

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,,,,
73c482b64,0.993520,0.030332,0.050700,0.941412,0.211176,0.403374,0.333553,0.461497,0.993698,0.147405,0.546766,0.131073,0.840620,0.689523,0.193913,0.092404,0.261579,0.047180,0.967313,0.083471,0.118150,0.237845,0.362608,0.621376
afaa8d38f,0.912487,0.303974,0.080591,0.733273,0.967078,0.279590,0.092508,0.217621,0.162663,0.079968,0.333501,0.133180,0.939886,0.709420,0.186118,0.102594,0.258104,0.076180,0.984060,0.085176,0.105579,0.617324,0.443138,0.608212
074cf066a,0.382550,0.282504,0.825998,0.096949,0.418305,0.157411,0.499742,0.973655,0.505856,0.151358,0.322229,0.995416,0.712397,0.665254,0.159914,0.827921,0.255147,0.100507,0.609599,0.670132,0.101762,0.273713,0.618289,0.095327
2a7255b0c,0.990820,0.923954,0.080412,0.562972,0.613984,0.241469,0.107155,0.259928,0.181540,0.250263,0.437851,0.159188,0.952830,0.498647,0.321196,0.036380,0.393738,0.076102,0.460789,0.078581,0.166773,0.410804,0.456178,0.758773
28ed37c9a,0.262467,0.331431,0.427942,0.867403,0.261143,0.929644,0.476801,0.821647,0.799290,0.154321,0.388407,0.990931,0.222905,0.072868,0.283256,0.984615,0.471285,0.114159,0.370742,0.235900,0.257989,0.411889,0.392797,0.949964
8bf90ebb9,0.207905,0.974910,0.194285,0.962516,0.134518,0.281184,0.225958,0.581694,0.152714,0.205003,0.340789,0.275335,0.153948,0.987981,0.393387,0.182120,0.482757,0.081616,0.279208,0.309832,0.294439,0.781051,0.396307,0.233212
a82802bd2,0.270048,0.617183,0.208113,0.977052,0.231861,0.214787,0.224366,0.196588,0.716294,0.435967,0.759182,0.257969,0.141012,0.736819,0.267157,0.654236,0.963476,0.214488,0.194228,0.160675,0.554177,0.671797,0.057749,0.229812
ed8b65c55,0.190514,0.290031,0.587511,0.707793,0.295910,0.441680,0.580033,0.358294,0.687417,0.761080,0.896416,0.401204,0.335101,0.510901,0.272128,0.287956,0.975598,0.758478,0.369418,0.670048,0.741869,0.519731,0.484186,0.519292
edf9fe22d,0.970775,0.116677,0.062330,0.860816,0.133922,0.500269,0.089466,0.311292,0.426235,0.082120,0.306377,0.103784,0.960740,0.740175,0.136954,0.118386,0.212054,0.033724,0.978291,0.076351,0.085508,0.231733,0.113551,0.636382


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
count,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000
mean,0.399510,0.412535,0.413781,0.775564,0.368171,0.427416,0.360654,0.513059,0.508412,0.444921,0.442136,0.374009,0.495155,0.566205,0.354265,0.480839,0.403470,0.213732,0.605932,0.378871,0.449714,0.478162,0.425217,0.424009
std,0.220499,0.309965,0.272892,0.262115,0.209531,0.214412,0.177136,0.286647,0.237535,0.235962,0.197482,0.264591,0.294546,0.230794,0.249950,0.287668,0.198584,0.180449,0.265974,0.228421,0.258789,0.198967,0.235755,0.260919
min,0.058366,0.027967,0.047624,0.059092,0.058343,0.074861,0.077205,0.069819,0.040817,0.066293,0.061473,0.073426,0.055047,0.047045,0.072994,0.032980,0.101726,0.031047,0.129321,0.059358,0.061618,0.159548,0.006015,0.033348
25%,0.255980,0.169184,0.166240,0.712257,0.204905,0.259519,0.226757,0.263318,0.336412,0.233231,0.304019,0.177695,0.225370,0.424864,0.174357,0.207602,0.241809,0.090215,0.377019,0.204995,0.210845,0.319064,0.236944,0.176005
50%,0.342214,0.290031,0.358319,0.903192,0.326205,0.385465,0.334500,0.398448,0.520534,0.422630,0.422127,0.274763,0.443334,0.631709,0.271426,0.479472,0.366630,0.154541,0.537914,0.315919,0.406538,0.439822,0.440134,0.419135
75%,0.451691,0.667685,0.629559,0.952873,0.478614,0.567205,0.470350,0.802006,0.684561,0.632402,0.558360,0.477543,0.758446,0.742203,0.437957,0.730889,0.515123,0.276974,0.914151,0.531087,0.694704,0.600606,0.593415,0.626573
max,0.993762,0.993202,0.995770,0.985371,0.994814,0.987151,0.989330,0.988314,0.995963,0.993637,0.992002,0.995416,0.990138,0.993667,0.994372,0.991921,0.995366,0.987852,0.989854,0.989765,0.995649,0.997606,0.997579,0.994196


In [4]:
sub.mean()

s0     0.413186
s1     0.400552
s2     0.423287
s3     0.811315
s4     0.391724
s5     0.464032
s6     0.389625
s7     0.492545
s8     0.457904
s9     0.491253
s10    0.472529
s11    0.385916
s12    0.537072
s13    0.515552
s14    0.364734
s15    0.481834
s16    0.301062
s17    0.222266
s18    0.752366
s19    0.367232
s20    0.516423
s21    0.414319
s22    0.518471
s23    0.418675
dtype: float64

In [5]:
sub.mean().mean()

0.05815756635761571

In [42]:
input = torch.tensor([[1,2,-1,-2]]).float()
target = torch.tensor([[1.,0.5,0,0]]).float()
input_differences = input.unsqueeze(1) - input.unsqueeze(2)
target_differences = target.unsqueeze(2) - target.unsqueeze(1)
exps = input_differences.exp() * torch.maximum(torch.tensor(0), target_differences)
lsep = torch.log(1 + exps.sum(2).sum(1))

In [50]:
xtorch.where(target_differences<0, torch.tensor(0.), target_differences)

tensor([[[0.0000, 0.5000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.5000, 0.5000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])

In [43]:
input_differences

tensor([[[ 0.,  1., -2., -3.],
         [-1.,  0., -3., -4.],
         [ 2.,  3.,  0., -1.],
         [ 3.,  4.,  1.,  0.]]])

In [44]:
target_differences

tensor([[[ 0.0000,  0.5000,  1.0000,  1.0000],
         [-0.5000,  0.0000,  0.5000,  0.5000],
         [-1.0000, -0.5000,  0.0000,  0.0000],
         [-1.0000, -0.5000,  0.0000,  0.0000]]])

In [45]:
exps

tensor([[[0.0000, 1.3591, 0.1353, 0.0498],
         [0.0000, 0.0000, 0.0249, 0.0092],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])

In [46]:
lsep

tensor([0.9471])